In [ ]:
import numpy as np
from math import exp, log

In [ ]:
def parity_matrix(m, n) :
    return 

In [ ]:
def non_zeros(M) :
    return 

In [ ]:
def syndrome(M1, x) :
    return np.matmul(M1, np.transpose(np.array(x)))

In [ ]:
def produce_a2f(p, Na, Nf, Ma, Mf) :
    a2f = [0]
    for i in range(1, Ma + 1) :
        a = exp(-i/Na)
        f = (1 + a)/(1 - a)
        z = log(f)
        j = int(Nf * z + 0.5) 
        if j <= 0 :
            j = 1
        if j > Mf :
            j = Mf
        a2f.append(j)
    retrun a2f

In [ ]:
def produce_f2a(p, Na, Nf, Ma, Mf) :
    f2a = [0]
    for i in range(1, Mf + 1) :
        f = exp(i/Nf)
        a = (f - 1)/(f + 1)
        z = -log(a)
        j = int(Na * z + 0.5) 
        if j <= 0 :
            j = 1
        if j > Ma :
            j = Ma
        f2a.append(j)
    retrun f2a

In [ ]:
def LL_init(Na, Nf, k, p) :
    LL_reg = [0]
    f_init = Nf * log((1-p)/p)
    a_init = Na * log(1 - 2 * p)
    for i in range(1, k + 1) :
        LL_reg.append(a_init)
    return f_init, LL_reg

In [ ]:
def cs_msgs_2_bits(m, d, cs_index, cs_list, LL_reg, Ma) : 
    for i in range(1, m + 1) :
        sign = d[i]
        big_alpha = 0
        j1 = cs_index[i]
        j2 = cs_index[i + 1]
        for j in range(j1, j2) :
            a1 = LL_reg[cs_list[j][1]]
            if a1 < 0 :
                sign = 1 - sign
                big_alpha -= a1
        for j in range(j1, j2) :
            a1 = LL_reg[cs_list[j][1]]
            p_sign = sign
            if a1 < 0 :
                p_alpha = big_alpha + a1
            else :
                p_alpha = big_alpha + a1
            if p_alpha <= 0 :
                p_alpha = 1
            if p_alpha > Ma :
                p_alpha = Ma
            if p_sign == 0 :
                LL_reg[cs_list[j][1]] = p_alpha
            else :
                LL_reg[cs_list[j][1]] = - p_alpha
    return LL_reg

In [ ]:
def bit_msgs_2_cs(n, f_init, a2f, Mf, LL_reg, y) :
    y1 = []
    for i in range(1, n + 1) :
        j1, j2 = LL_index[i], LL_index[i + 1]
        f_tot = f_init
        for j in range(j1, j2) :
            u = LL_reg[j]
            if u > 0 :
                u = a2f[u]
            else :
                u = - a2f[-u]
            LL_reg[j] = u
            f_tot += u
        
        for j in range(j1, j2) :
            x = f_tot - LL_reg[j]
            if x < 0 :
                p_sign = 1
                x = -x
            else :
                p_sign = 0
            if x < 1 :
                x = 1
            if x > Mf :
                x = Mf
            if p_sign == 1 :
                LL_reg[j] = - f2a[x]
            else :
                LL_reg[j] = f2a[x]
    
        if f_tot < 0 :
            y1.append(1 - y[i])
        else : 
            y1.append(y[i])
        return y1, LL_reg

In [ ]:
def converged(m, cs_index, cs_list, c) :
    success = 1
    for i in range(1, m + 1) :
        chksum = 0
        j1, j2 = cs_index[i], cs_index[i + 1]
        for j in range(j1, j2) :
            chksum = chksum ^ cs_list[j][0]
        if chksum != c[i] :
            success = 0
    return success
        

In [ ]:
def belief_prop(C, D, MAX_ITERS, Na, Nf, Ma, Mf, p, k) :
    a2f, f2a = produce_a2f(p, Na, Nf, Ma, Mf), produce_f2a(p, Na, Nf, Ma, Mf)
    f_init, LL_reg = LL_init(Na, Nf, k, p)
    success = 0
    i = 0
    while (i < MAX_ITERS) :
        LL_reg = cs_msgs_2_bits(m, d, cs_index, cs_list, LL_reg, Ma)
        y, LL_reg = bit_msgs_2_cs(n, f_init, a2f, Mf, LL_reg, y)
        success = converged(m, cs_index, cs_list, c)
        i += 1
        if success == 1 : 
            break
    return y, success

In [ ]:
#Step1 : Produce parity check matrix(M1) of dimension m*n, n is length of bitstring and m is no. of parity check equations
M1 = parity_matrix(m, n)
k = non_zeros(M)

#Step2 : Sita produces the syndrome and hash values
C = syndrome(M1, sita_bits)
H = hash_function(sita_bits)

#Step3 : Sita sends syndrome and hash values via CAC

#Step4 : Ram produces the syndrome
D = syndrome(M1, ram_bits)

#Step5 : Ram performs belief propagation algorithm
y1, success = belief_prop(C, D, MAX_ITERS, Na, Nf, Ma, Mf, p, k)
    
#Step6 : Ram sends success of reconciliation
    
    